In [63]:
!pip install flwr

In [44]:
import pandas as pd, glob, base64, json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import flwr as fl, matplotlib.pyplot as plt, seaborn as sns
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import tensorflow.keras.backend as K
from tensorflow.keras.layers import (Input, Dense, Dropout, BatchNormalization,
                                     Lambda)
from tensorflow.keras.models import Model, Sequential


In [64]:
# ---------- Custom Scaffold strategy (server side) ----------
import flwr as fl
from typing import Callable, Dict, List, Optional, Tuple
import numpy as np

class Scaffold(fl.server.strategy.FedAvg):
    """Scaffold implementation – control-variates with full-client participation."""

    def __init__(
        self,
        *,
        fraction_fit: float = 1.0,
        min_fit_clients: int = 2,
        min_available_clients: int = 2,
        on_fit_config_fn: Optional[Callable] = None,
        server_learning_rate: float = 1.0,
    ):
        super().__init__(
            fraction_fit=fraction_fit,
            min_fit_clients=min_fit_clients,
            min_available_clients=min_available_clients,
            on_fit_config_fn=on_fit_config_fn,
        )
        self.server_lr = server_learning_rate
        self.c = None  # global control variate

    # --------------------------------------------------------------------- #
    def initialize_parameters(self, client_manager):
        params = super().initialize_parameters(client_manager)
        self.c = [np.zeros_like(p) for p in params.tensors]
        return params

    # --------------------------------------------------------------------- #
    def configure_fit(
        self, rnd: int, parameters, client_manager
    ):
        cfg, fit_ins = super().configure_fit(rnd, parameters, client_manager)
        # Append server control variate to config
        fit_ins = [
            (cid, fl.common.FitIns(parameters, {"c_global": self.c})) for cid, _ in cfg
        ]
        return fit_ins, []

    # --------------------------------------------------------------------- #
    def aggregate_fit(self, rnd, results, failures):
        if failures:
            raise RuntimeError("No partial failure handling for Scaffold")

        # Average normal FedAvg style
        new_params, _ = super().aggregate_fit(rnd, results, failures)

        # Aggregate client deltas for control variate update
        deltas = [
            [
                np.subtract(p_old, p_new) / (fit_res.metrics["num_examples"])
                for p_old, p_new in zip(self.parameters.tensors, res.parameters.tensors)
            ]
            for (_, fit_res) in results
        ]
        delta_mean = [
            np.mean([d[layer] for d in deltas], axis=0) for layer in range(len(self.c))
        ]
        # Update global control variate
        self.c = [c_i + delta_i for c_i, delta_i in zip(self.c, delta_mean)]

        # Server model move opposite to control variate (as in paper)
        new_tensors = [
            w - self.server_lr * c for w, c in zip(new_params.tensors, self.c)
        ]
        self.parameters = fl.common.ndarrays_to_parameters(new_tensors)
        return self.parameters, {}


In [65]:
# ---------- Custom Scaffold client wrapper ----------
import tensorflow as tf

class ScaffoldClient(fl.client.NumPyClient):
    def __init__(self, model_fn, X, y, cid):
        self.model = model_fn()
        self.X, self.y = X, y
        self.cid = cid
        # local control variate
        self.ci = [np.zeros_like(w) for w in self.model.get_weights()]

    # ------------- Flower API -------------------------------------------- #
    def get_parameters(self, config):          # not used
        return self.model.get_weights()

    def fit(self, parameters, config):
        # Set global weights
        self.model.set_weights(parameters)
        # Get server control variate
        c_global = config["c_global"]

        # Apply Scaffold correction on weights
        corrected_weights = [
            w - c_g + c_i for w, c_g, c_i in zip(self.model.get_weights(), c_global, self.ci)
        ]
        self.model.set_weights(corrected_weights)

        self.model.fit(self.X, self.y, epochs=1, batch_size=64, verbose=0)

        # Compute weight updates
        new_weights = self.model.get_weights()
        delta_ci = [c_g - c_i + (w_old - w_new) for w_old, w_new, c_g, c_i
                    in zip(corrected_weights, new_weights, c_global, self.ci)]
        self.ci = [c_i + delta for c_i, delta in zip(self.ci, delta_ci)]

        return new_weights, len(self.X), {"num_examples": len(self.X)}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        return loss, len(self.y), {"accuracy": acc}


In [45]:
SEED = 42

In [46]:
DATA_PATHS = {
    "iot23": "/kaggle/input/iot23-dataset",
    "unsw" : "/kaggle/input/unsw-nb15",
    "cic17": "/kaggle/input/network-intrusion-dataset",
    "cic18": "/kaggle/input/ids-intrusion-csv"
}
LABEL_COL = {
    "iot23": "label",        # <── küçük l
    "unsw" : "label",
    "cic17": "Label",
    "cic18": "Label"
}
BENIGN = {
    "iot23": "Benign",       # IoT-23’te benign’in B’si yine büyük
    "unsw" : "Normal",
    "cic17": "BENIGN",
    "cic18": "Benign"
}

def _read_csv_safe(path, nrows=None):
    """Farklı kodlamaları sırayla dene; başarıya ilk ulaşanı döndür."""
    for enc in ("utf-8", "utf-8-sig", "latin1", "cp1252"):
        try:
            return pd.read_csv(path, nrows=nrows, low_memory=False,
                               encoding=enc, on_bad_lines="skip")
        except UnicodeDecodeError:
            continue   # sıradaki kodlamayı dene
    # hepsi çökerse son çare: python motoru + sorunlu byte'ları atla
    return pd.read_csv(path, nrows=nrows, low_memory=False, engine="python",
                       encoding="latin1", on_bad_lines="skip", encoding_errors="ignore")

def load_dataset(key, nrows=None):
    path, label, benign = DATA_PATHS[key], LABEL_COL[key], BENIGN[key]

    # --- CSV oku -----------------------------------------------------------
    frames = [_read_csv_safe(p, nrows) for p in glob.glob(f"{path}/*.csv")]
    df = pd.concat(frames, ignore_index=True)

    df.columns = df.columns.str.strip()          #  <<<  BU SATIR YENİ   <<<

    # ----------------------------------------------------------------------
    num_df = (df.select_dtypes(include="number")
                .fillna(0).replace([np.inf, -np.inf], 0)
                .astype("float32"))
    X = num_df.values
    y = (df[label] != benign).astype("int32").values

    # --- split & scale ------------------------------------------------------
    Xtr, Xte, ytr, yte = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=SEED
    )
    scaler = StandardScaler().fit(Xtr)
    Xtr = scaler.transform(Xtr).astype("float32")
    Xte = scaler.transform(Xte).astype("float32")
    return Xtr, Xte, ytr, yte

In [48]:
class CryptographicUtils:
    @staticmethod
    def gen_key(pwd:str)->bytes:
        salt=b"salt_secure_kd"
        kdf=PBKDF2HMAC(algorithm=hashes.SHA256(),length=32,salt=salt,iterations=100_000)
        return base64.urlsafe_b64encode(kdf.derive(pwd.encode()))
    @staticmethod
    def encrypt(weights:dict,key:bytes)->dict:
        f=Fernet(key); out={}
        for name,w in weights.items():
            out[name]={"data":f.encrypt(w.tobytes()),
                       "shape":w.shape,"dtype":str(w.dtype)}
        return out
    @staticmethod
    def decrypt(enc:dict,key:bytes)->dict:
        f=Fernet(key); out={}
        for name,info in enc.items():
            arr=np.frombuffer(f.decrypt(info["data"]),dtype=info["dtype"])
            out[name]=arr.reshape(info["shape"])
        return out
    @staticmethod
    def add_dp_noise(w,eps=1.0,delta=1e-5):
        sigma=np.sqrt(2*np.log(1.25/delta))/eps
        return w+np.random.normal(0,sigma,w.shape)

class SecureAggregator:
    def __init__(self,num_clients:int): self.num_clients=num_clients
    def aggregate(self,updates:list,sizes:list)->dict:
        tot=sum(sizes); agg={}
        for upd,sz in zip(updates,sizes):
            factor=sz/tot
            for k,v in upd.items():
                agg[k]=agg.get(k,np.zeros_like(v))+v*factor
        return agg


In [49]:
class FedVAEKD:
    def __init__(self,num_clients:int,pwd:str="secure2025",dp_eps=1.0):
        self.num_clients=num_clients
        self.key=CryptographicUtils.gen_key(pwd)
        self.aggregator=SecureAggregator(num_clients)
        self.clients={}
        self.comm_costs=[]
        self.dp_eps=dp_eps
    def add_client(self,cid,X,y): self.clients[cid]={"X":X,"y":y,"n":len(X)}
    # --------- model tanımları ----------
    def _vae(self,in_dim,inter=128,lat=64):
        inp=Input((in_dim,))
        h=BatchNormalization()(Dense(inter,activation="relu")(inp))
        h=Dropout(0.2)(h)
        z_mean=Dense(lat)(h); z_log=Dense(lat)(h)
        z=Lambda(lambda a: a[0]+K.random_normal(K.shape(a[0]))*K.exp(0.5*a[1]))([z_mean,z_log])
        dh=BatchNormalization()(Dense(inter,activation="relu")(z))
        out=Dense(in_dim,activation="sigmoid")(dh)
        vae=Model(inp,out); enc=Model(inp,z_mean)
        rec=tf.reduce_mean(tf.reduce_sum(tf.keras.losses.binary_crossentropy(inp,out),-1))
        kl=-0.5*tf.reduce_mean(tf.reduce_sum(1+z_log-K.square(z_mean)-K.exp(z_log),-1))
        vae.add_loss(rec+1.5*kl); vae.compile("adam")
        return vae,enc
    def _teacher(self,lat_dim,classes):
        m=Sequential([Dense(256,activation="relu",input_shape=(lat_dim,)),
                      BatchNormalization(),Dropout(0.3),
                      Dense(128,activation="relu"),BatchNormalization(),
                      Dropout(0.3),Dense(64,activation="relu"),
                      BatchNormalization(),Dropout(0.2),
                      Dense(classes,activation="softmax")])
        m.compile("adam","sparse_categorical_crossentropy",["accuracy"])
        return m
    # --------- tek round ----------
    def round(self,epochs=3,dp=True):
        enc_updates,sizes,comm=[],[],0
        for cid,d in self.clients.items():
            vae,enc=self._vae(d["X"].shape[1])
            if dp:
                for l in vae.layers:
                    ws=l.get_weights()
                    if ws: l.set_weights([CryptographicUtils.add_dp_noise(ws[0],self.dp_eps)]+ws[1:])
            vae.fit(d["X"],epochs=epochs,batch_size=32,verbose=0)
            w={f"l{i}":l.get_weights()[0] for i,l in enumerate(vae.layers) if l.get_weights()}
            enc_w=CryptographicUtils.encrypt(w,self.key)
            comm+=sum(len(v["data"]) for v in enc_w.values())
            enc_updates.append(CryptographicUtils.decrypt(enc_w,self.key))
            sizes.append(d["n"])
        self.comm_costs.append(comm)
        return self.aggregator.aggregate(enc_updates,sizes)


In [77]:
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

def build_student(in_dim):
    """Uyarısız, dtype-safe küçük model."""
    model = Sequential([
        Input(shape=(in_dim,), dtype="float32"),        # <-- eklendi
        Dense(128, activation="relu"),
        BatchNormalization(), Dropout(0.3),
        Dense(64, activation="relu"),
        BatchNormalization(), Dropout(0.3),
        Dense(2, activation="softmax")
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

class FlowerClient(fl.client.NumPyClient):
    def __init__(self,X,y,in_dim): self.X,self.y,self.model=X,y,build_student(in_dim)
    def get_parameters(self,config): return self.model.get_weights()
    def fit(self,params,config):
        self.model.set_weights(params)
        self.model.fit(self.X,self.y,epochs=config["local_epochs"],batch_size=64,verbose=0)
        return self.model.get_weights(),len(self.X),{}
    def evaluate(self,params,config):
        self.model.set_weights(params)
        l,a=self.model.evaluate(self.X,self.y,verbose=0)
        return l,len(self.X),{"accuracy":a}

def simulate_fl(method, Xtr, ytr, Xte, yte,
                clients=10, rounds=3, client_resources=None):

    # ---------------- dataset parçala ----------------
    idx = np.random.permutation(len(Xtr))
    parts = [(Xtr[s], ytr[s]) for s in np.array_split(idx, clients)]

    def model_fn():                     # küçük Keras model
        return build_student(Xtr.shape[1])

    def client_fn(cid):
        i = int(cid)
        Xc, yc = parts[i]
        if method == "scaffold":
            return ScaffoldClient(model_fn, Xc, yc, i)
        return FlowerClient(Xc, yc, Xtr.shape[1])

    # ---------------- strateji seç -------------------
    if method == "scaffold":
        strategy = Scaffold(
            min_fit_clients=clients,
            min_available_clients=clients,   #  <<< uyarı kalkar
            fraction_fit=1.0,
            on_fit_config_fn=lambda r: {}
        )
    elif method == "fedprox":
        strategy = fl.server.strategy.FedProx(
            proximal_mu=0.01,
            min_fit_clients=clients,
            min_available_clients=clients,
            fraction_fit=1.0,
            on_fit_config_fn=lambda r: {}
        )
    elif method == "fedadam":
        strategy = fl.server.strategy.FedAdam(
            min_fit_clients=clients,
            min_available_clients=clients,
            fraction_fit=1.0,
            on_fit_config_fn=lambda r: {}
        )
    else:  # fedavg
        strategy = fl.server.strategy.FedAvg(
            min_fit_clients=clients,
            min_available_clients=clients,
            fraction_fit=1.0,
            on_fit_config_fn=lambda r: {}
        )

    # ---------------- Flower sim ---------------------
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=clients,
        config=fl.server.ServerConfig(num_rounds=rounds),
        client_resources=client_resources or {"num_cpus":2},
        strategy=strategy,
    )

    # -------------- global model test ----------------
    global_model = model_fn()
    global_model.set_weights(strategy.parameters.tensors)
    preds = np.argmax(global_model.predict(Xte, verbose=0), 1)
    return {"accuracy": accuracy_score(yte, preds),
            "f1": f1_score(yte, preds)}


In [51]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def evaluate_central(Xtr, ytr, Xte, yte, epochs=5):
    # Cast’ler ile hiçbir şüphe kalmıyor
    Xtr = Xtr.astype("float32", copy=False)
    Xte = Xte.astype("float32", copy=False)
    ytr = ytr.astype("int32",  copy=False)
    yte = yte.astype("int32",  copy=False)

    model = build_student(Xtr.shape[1])
    model.fit(Xtr, ytr, epochs=epochs, batch_size=128, verbose=0)

    preds = np.argmax(model.predict(Xte, verbose=0), axis=1)
    return {
        "accuracy": accuracy_score(yte, preds),
        "f1":       f1_score(yte, preds)
    }


def barplot(results,title):
    df=pd.DataFrame(results).T.reset_index()
    df.columns=["dataset","method","accuracy","f1"]
    plt.figure(figsize=(12,5))
    sns.barplot(df,x="method",y="f1",hue="dataset"); plt.title(title); plt.xticks(rotation=45); plt.show()


In [52]:
# (Hücre 3’ten sonra tek seferlik kontrol)
for ds in DATA_PATHS:
    Xtr, Xte, ytr, yte = load_dataset(ds, nrows=10_000)   # küçük parça
    print(f"{ds}:  X dtype={Xtr.dtype}, y dtype={ytr.dtype}, X shape={Xtr.shape}")


iot23:  X dtype=float32, y dtype=int32, X shape=(16000, 26)
unsw:  X dtype=float32, y dtype=int32, X shape=(48205, 163)
cic17:  X dtype=float32, y dtype=int32, X shape=(64000, 78)
cic18:  X dtype=float32, y dtype=int32, X shape=(80000, 1)


In [53]:
import tensorflow as tf, os
print(tf.__version__)
print("GPU list:", tf.config.list_physical_devices("GPU"))
!nvidia-smi -L

2.18.0
GPU list: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-394567c8-e99f-2602-b4d7-d2a5390c29f0)


In [78]:
from tqdm.auto import tqdm

ALL = {}
for ds in tqdm(DATA_PATHS, desc="Datasets"):
    Xtr,Xte,ytr,yte = load_dataset(ds)

    ALL[(ds,"central")] = evaluate_central(Xtr,ytr,Xte,yte)

    for m in tqdm(["fedavg","fedprox","fedadam","scaffold"],
                  desc=f"{ds} strategies", leave=False):
        ALL[(ds,m)] = simulate_fl(
            m, Xtr, ytr, Xte, yte,
            rounds=3,
            client_resources={"num_cpus":2,"num_gpus":0.25}
        )


Datasets:   0%|          | 0/4 [00:00<?, ?it/s]

iot23 strategies:   0%|          | 0/4 [00:00<?, ?it/s]

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
[2025-06-05 10:54:40,828 W 35 35] gcs_rpc_client.h:151: Failed to connect to GCS at address 172.19.2.2:50043 within 5 seconds.
[2025-06-05 10:55:10,832 W 35 35] gcs_client.cc:182: Failed to get cluster ID from GCS server: TimedOut: Timed out while waiting for GCS to become available.


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/ray/session_2025-06-05_10-54-35_742758_35/logs/gcs_server.err'

# SECOND TURN WITHOUT ANY PORT

In [81]:
import numpy as np, tensorflow as tf, copy
from sklearn.metrics import accuracy_score, f1_score

In [82]:
def model_weights(model):
    return [w.numpy() for w in model.weights]

def set_model_weights(model, weights):
    for var, w in zip(model.weights, weights):
        var.assign(w)

def train_local(model, X, y, epochs=1):
    model.fit(X, y, epochs=epochs, batch_size=64, verbose=0)
    return model


In [83]:
class FedStrategy:
    def __init__(self, lr_server=1.0, mu=0.0):      # mu: FedProx
        self.lr_server = lr_server
        self.mu = mu
        self.state = {}

    def aggregate(self, client_weights, client_sizes):
        tot = sum(client_sizes)
        return [sum(w*c/ tot for w, c in zip(layer, client_sizes))
                for layer in zip(*client_weights)]

    def server_update(self, global_w, agg_w):        # FedAvg / Prox / Adam
        return agg_w                                # default = FedAvg

class FedAdam(FedStrategy):
    def __init__(self, beta1=0.9, beta2=0.999, eps=1e-8, lr_server=0.001):
        super().__init__(lr_server)
        self.m, self.v, self.t = None, None, 0
        self.beta1, self.beta2, self.eps = beta1, beta2, eps

    def server_update(self, global_w, agg_w):
        if self.m is None:
            self.m = [np.zeros_like(w) for w in global_w]
            self.v = [np.zeros_like(w) for w in global_w]
        self.t += 1
        g = [gw - w for gw, w in zip(global_w, agg_w)]
        self.m = [self.beta1*m + (1-self.beta1)*gi for m, gi in zip(self.m, g)]
        self.v = [self.beta2*v + (1-self.beta2)*gi*gi for v, gi in zip(self.v, g)]
        m_hat = [m/(1-self.beta1**self.t) for m in self.m]
        v_hat = [v/(1-self.beta2**self.t) for v in self.v]
        return [w - self.lr_server*mh/(np.sqrt(vh)+self.eps)
                for w, mh, vh in zip(global_w, m_hat, v_hat)]

class FedProx(FedStrategy):
    pass                                           # server side = FedAvg

class Scaffold(FedStrategy):
    def __init__(self, lr_server=1.0):
        super().__init__(lr_server)
        self.c_global = None

    def server_update(self, global_w, agg_w):
        if self.c_global is None:
            self.c_global = [np.zeros_like(w) for w in global_w]
        # Scaffold server update (FedAvg step – lr*cg)
        return [w_g - self.lr_server*cg for w_g, cg in zip(agg_w, self.c_global)]


In [84]:
def scaffold_local_train(model, X, y, c_global, c_local, epochs=1):
    """Return (new_weights, new_c_local)."""
    w_before = model_weights(model)
    # Adjust weights before training
    set_model_weights(model, [w - c_g + c_l for w, c_g, c_l
                              in zip(w_before, c_global, c_local)])
    model.fit(X, y, epochs=epochs, batch_size=64, verbose=0)
    w_after = model_weights(model)
    # Update local control variate
    delta = [wb - wa for wb, wa in zip(w_before, w_after)]
    new_c = [c_l - c_g + d/epochs for c_l, c_g, d in zip(c_local, c_global, delta)]
    return w_after, new_c


In [85]:
def mini_simulate(strategy_name, Xtr, ytr, Xte, yte,
                  clients=5, rounds=3, epochs=1):
    # ---------- Böl PARÇALA ----------
    idx = np.random.permutation(len(Xtr))
    parts = [(Xtr[s], ytr[s]) for s in np.array_split(idx, clients)]

    def new_model(): return build_student(Xtr.shape[1])

    # ---------- Strateji seç ----------
    if strategy_name == "fedavg":
        strat = FedStrategy()
    elif strategy_name == "fedprox":
        strat = FedProx(mu=0.01)
    elif strategy_name == "fedadam":
        strat = FedAdam()
    elif strategy_name == "scaffold":
        strat = Scaffold(lr_server=1.0)
    else:
        raise ValueError("unknown strategy")

    # ---------- Başlat ----------
    global_model = new_model()
    global_w = model_weights(global_model)
    c_locals = [[np.zeros_like(w) for w in global_w] for _ in range(clients)]

    # ---------- Eğitim döngüsü ----------
    for _ in range(rounds):
        client_weights, client_sizes = [], []
        for cid, (Xc, yc) in enumerate(parts):
            local_model = new_model()
            set_model_weights(local_model, copy.deepcopy(global_w))

            if strategy_name == "scaffold":
                w_new, c_new = scaffold_local_train(
                    local_model, Xc, yc,
                    strat.c_global or [np.zeros_like(w) for w in global_w],
                    c_locals[cid], epochs)
                c_locals[cid] = c_new
            else:
                train_local(local_model, Xc, yc, epochs)
                w_new = model_weights(local_model)

            client_weights.append(w_new)
            client_sizes.append(len(Xc))

        agg_w = strat.aggregate(client_weights, client_sizes)
        global_w = strat.server_update(global_w, agg_w)
        set_model_weights(global_model, global_w)
        if strategy_name == "scaffold":
            strat.c_global = [ np.mean([c[i] for c in c_locals], axis=0)
                               for i in range(len(global_w)) ]

    # ---------- Değerlendir ----------
    preds = np.argmax(global_model.predict(Xte, verbose=0), axis=1)
    return {"accuracy": accuracy_score(yte, preds),
            "f1": f1_score(yte, preds)}


In [ ]:
from tqdm.auto import tqdm
ALL={}
for ds in tqdm(DATA_PATHS, desc="Datasets"):
    Xtr,Xte,ytr,yte = load_dataset(ds)

    ALL[(ds,"central")] = evaluate_central(Xtr,ytr,Xte,yte)

    for strat in tqdm(["fedavg","fedprox","fedadam","scaffold"],
                      desc=f"{ds} strategies", leave=False):
        ALL[(ds,strat)] = mini_simulate(
            strat, Xtr, ytr, Xte, yte,
            clients=5, rounds=3, epochs=1
        )
    # FedVAEKD hâlâ isteğe bağlı


Datasets:   0%|          | 0/4 [00:00<?, ?it/s]

iot23 strategies:   0%|          | 0/4 [00:00<?, ?it/s]